<a href="https://colab.research.google.com/github/AQadir44/REGRESSION_PROJECTS/blob/master/housing_price_regression_h2o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install h2o

     |████████████████████████████████| 126.2MB 55kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.3-py2.py3-none-any.whl size=126303006 sha256=15e0e936fc09aac7085178920bca67f55afb0250bb2420cd86a81a949c795e54
  Stored in directory: /root/.cache/pip/wheels/fa/7c/ce/95ae52b4d3f1b14a27c3c961c1f94635aee841ab1eec3aeeca
Successfully built h2o


In [3]:
! apt-get install default-jre
!java -version

Reading package lists... Done
Building dependency tree       
Reading state information... Done
default-jre is already the newest version (2:1.11-68ubuntu1~18.04.1).
default-jre set to manually installed.
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 25 not upgraded.
openjdk version "11.0.6" 2020-01-14
OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1)
OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)


In [0]:
import numpy as np    # linear algebra
import pandas as pd 

In [0]:
import h2o as h
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators import H2ORandomForestEstimator

In [61]:
## Import the dataset

train = pd.read_csv("../content/train.csv")
test = pd.read_csv("../content/test.csv")
test_original = test
print(train.shape)
print(train.columns)

(1460, 81)
Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 

In [62]:
## Clean the dataset



# Identify the columns containing NA values in both train and test dataset
train.isnull().sum().sort_values(ascending = False)
test.isnull().sum().sort_values(ascending = False)
# Remove the columns which contain NA with more than 100 rows
train.drop( ['PoolQC', 'MiscFeature','Alley','Fence','FireplaceQu','LotFrontage'], inplace = True, axis = 'columns')
test.drop( ['PoolQC', 'MiscFeature','Alley','Fence','FireplaceQu','LotFrontage'], inplace = True, axis = 'columns')
print(train.shape)
print(test.shape)
# Identify the columns which contain numeric values
train_numericCol = train.select_dtypes(include=[np.number]).columns.values
print(train_numericCol)
# Fill missing values in the numeric columns.
train.fillna(train.mean(),inplace = True)
test.fillna(test.mean(),inplace = True)
# check if still any NA's are available in the numerical coulumns of train data
train[train_numericCol].isnull().sum().sort_values(ascending = False)

(1460, 75)
(1459, 74)
['Id' 'MSSubClass' 'LotArea' 'OverallQual' 'OverallCond' 'YearBuilt'
 'YearRemodAdd' 'MasVnrArea' 'BsmtFinSF1' 'BsmtFinSF2' 'BsmtUnfSF'
 'TotalBsmtSF' '1stFlrSF' '2ndFlrSF' 'LowQualFinSF' 'GrLivArea'
 'BsmtFullBath' 'BsmtHalfBath' 'FullBath' 'HalfBath' 'BedroomAbvGr'
 'KitchenAbvGr' 'TotRmsAbvGrd' 'Fireplaces' 'GarageYrBlt' 'GarageCars'
 'GarageArea' 'WoodDeckSF' 'OpenPorchSF' 'EnclosedPorch' '3SsnPorch'
 'ScreenPorch' 'PoolArea' 'MiscVal' 'MoSold' 'YrSold' 'SalePrice']


SalePrice        0
BsmtHalfBath     0
GrLivArea        0
LowQualFinSF     0
2ndFlrSF         0
1stFlrSF         0
TotalBsmtSF      0
BsmtUnfSF        0
BsmtFinSF2       0
BsmtFinSF1       0
MasVnrArea       0
YearRemodAdd     0
YearBuilt        0
OverallCond      0
OverallQual      0
LotArea          0
MSSubClass       0
BsmtFullBath     0
FullBath         0
YrSold           0
HalfBath         0
MoSold           0
MiscVal          0
PoolArea         0
ScreenPorch      0
3SsnPorch        0
EnclosedPorch    0
OpenPorchSF      0
WoodDeckSF       0
GarageArea       0
GarageCars       0
GarageYrBlt      0
Fireplaces       0
TotRmsAbvGrd     0
KitchenAbvGr     0
BedroomAbvGr     0
Id               0
dtype: int64

In [63]:
h.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 0 mins
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,"14 days, 19 hours and 52 minutes"
H2O cluster name:,H2O_from_python_unknownUser_ni12em
H2O cluster total nodes:,1
H2O cluster free memory:,2.999 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"locked, healthy"


In [64]:
## Convert data into h2o Frame


train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)
test_original = h2o.H2OFrame(test_original)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [65]:
# Split the train dataset
train, valid, test = train.split_frame(ratios=[0.7, 0.15], seed=42)

# Seperate the target data and store it into y variable
y = 'SalePrice'
Id = test['Id']

# remove target and Id column from the dataset and store rest of the columns in X variable
X = list(train.columns)
X.remove(y)
X.remove('Id')
X

['MSSubClass',
 'MSZoning',
 'LotArea',
 'Street',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'OverallQual',
 'OverallCond',
 'YearBuilt',
 'YearRemodAdd',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'MasVnrArea',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinSF1',
 'BsmtFinType2',
 'BsmtFinSF2',
 'BsmtUnfSF',
 'TotalBsmtSF',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 '1stFlrSF',
 '2ndFlrSF',
 'LowQualFinSF',
 'GrLivArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'FullBath',
 'HalfBath',
 'BedroomAbvGr',
 'KitchenAbvGr',
 'KitchenQual',
 'TotRmsAbvGrd',
 'Functional',
 'Fireplaces',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageCars',
 'GarageArea',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'WoodDeckSF',
 'OpenPorchSF',
 'EnclosedPorch',
 '3SsnPorch',
 'ScreenPorch',

In [0]:
## H2o Machine Learning models

#We will now perform training of the models using below H2o supervised algorithms


#Gradient Boosting Machine (RF)

#Random Forest (RF)

#Deep Learning (DL)


# Prepare the hyperparameters
nfolds = 5
rf_params = {
                'max_depth': [3, 4,5],
                'sample_rate': [0.8, 1.0],               # Row sample rate
                'mtries' : [2,4,3]
                }

In [0]:
from h2o.grid.grid_search import H2OGridSearch
 # Search criteria for parameter space
search_criteria = {'strategy': "RandomDiscrete",
                   "seed": 1,
                   'stopping_metric': "AUTO",
                   'stopping_tolerance': 0.0005
                   }



# Prepare the grid object
rf_grid = H2OGridSearch(model=H2ORandomForestEstimator,   # Model to be trained
                          grid_id='rf_grid',                  # Grid Search ID
                          hyper_params=rf_params,              # Dictionary of parameters
                          search_criteria=search_criteria,   # RandomDiscrete
                          )
                          

In [73]:
# Train the Model

rf_grid.train(x=X,y=y, 
                training_frame=train,
                validation_frame=valid,
                ntrees=100,      
                score_each_iteration=True,
                nfolds = nfolds,
                fold_assignment= "Modulo",
                seed=1
                )




drf Grid Build progress: |████████████████████████████████████████████████| 100%


In [74]:
# Find the Model performance 
rf_gridperf = rf_grid.get_grid(sort_by='RMSE',decreasing = False)
rf_gridperf

     max_depth mtries sample_rate         model_ids                rmse
0            5      4         0.8   rf_grid_model_7    36218.4543011059
1            5      4         1.0   rf_grid_model_3  36322.701651330164
2            5      3         1.0   rf_grid_model_8   37957.18816271537
3            5      3         0.8  rf_grid_model_17   38056.94220517328
4            4      4         1.0  rf_grid_model_10   38965.50780705376
5            4      4         0.8   rf_grid_model_4   39131.66835440497
6            4      3         0.8  rf_grid_model_13  40918.251546480074
7            4      3         1.0   rf_grid_model_6   41025.15395980535
8            5      2         1.0   rf_grid_model_9   41256.15075160859
9            5      2         0.8  rf_grid_model_18    41383.2796039889
10           3      4         1.0  rf_grid_model_14   43065.61022488983
11           3      4         0.8   rf_grid_model_1   43182.28901524526
12           4      2         1.0   rf_grid_model_5   44342.4765

In [0]:
best_rf_model = rf_gridperf.models[0]
best_rf_model
best_rf_perf = best_rf_model.model_performance(test)   # Random Forest Model

In [77]:
rf_pred = best_rf_model.predict(test_original).as_data_frame()

drf prediction progress: |████████████████████████████████████████████████| 100%


/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'MSZoning' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Utilities' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior1st' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Exterior2nd' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'KitchenQual' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3.6/dist-packages/h2o/job.py:70: UserWarning: Test/Validation dataset column 'Functional' has levels not trained on: [nan]
  warnings.warn(w)
/usr/local/lib/python3

In [0]:
sub = pd.DataFrame()
sub['Id'] = rf_pred.index + 1461
sub['SalePrice'] = rf_pred
sub.head()
sub.to_csv('gbm_h2o1.csv', index=False)
